In [18]:
import pandas as pd
import dask.dataframe as dd
import os
import json
import dask
import redis

redis_client = redis.Redis(host="localhost", port=6378)

from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
client = cluster.get_client()

csv_df = pd.read_csv('/home/ubuntu/Data_Science/Big_Data/dask-lab/pokemonDB_dataset.csv')

parquet_df = dd.read_parquet('/home/ubuntu/Data_Science/Big_Data/dask-lab/trainers_with_pockemon.parquet')

mean_def = csv_df['Defense Max'].mean()

filter_df = csv_df[csv_df['Defense Max'] > mean_def]

merge_df = parquet_df.merge(filter_df)

merge_df = merge_df.drop(['pockemon'], axis=1)

merge_df['team_name'] = merge_df['first_name'] + " " + merge_df['last_name']

merge_df = merge_df.drop(['first_name', 'last_name'], axis=1)

high_def_team = merge_df.groupby(['team_name']).agg({'Defense Max': 'mean'})

h = high_def_team.persist()
h_computed = h.compute().reset_index()  # Reset the index

connection = redis_client

records = h_computed.to_dict(orient='records')  

for record in records:
    team_name = record['team_name']
    defense_mean = record['Defense Max']
    connection.set(team_name, json.dumps(defense_mean))


/home/ubuntu/.cache/pypoetry/virtualenvs/data-science-KLwnPlAe-py3.10/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33051 instead
  warnings.warn(


In [23]:
print(connection.keys("A*")) 



[b'Art Sandoval', b'Alane Carey', b'Alise Kelly', b'Antoine Murphy', b'Ai Holcomb', b'Andrea Bray', b'Angle Stuart', b'Analisa Carpenter', b'Arnold Gentry', b'Alica Downs', b'Alonzo Herrera', b'Annett Christensen', b'Andy Carey', b'Annelle Huber', b'Altha Haynes', b"Anika O'neill", b'Arletha Hickman', b'Anja Pittman', b'Asa Drake', b'Analisa Mccray', b'Alane Estes', b'Angila Beasley', b'Antonio Klein', b'An Savage', b'Aaron Fitzpatrick', b'Ayanna Jordan', b'Ayana Bryan', b'Alisia Fulton', b'Arnulfo Snyder', b'Ariane Mcgowan', b'Assunta Best', b'Andrew Parsons', b'Anjanette Aguirre', b'Annalisa Welch', b'Annice Rivera', b'Adalberto Mooney', b'Almeta Daniels', b'Arie Chaney', b'Amee Stevenson', b'Agnus Potter', b'Anastacia Pugh', b'Arturo Roach', b'Alfredo Vasquez', b'Alden Hodge', b'Agustin Buckley', b'Alejandrina Lopez', b'Ashley Freeman', b'Aaron Owens', b'An Valenzuela', b'Antwan Weeks', b'Arnulfo Gates', b'Austin Bell', b'Annalisa Holloway', b'Audria Woods', b'Anjanette Bright', b'A

In [24]:
print(connection.get("Andrea Bray"))

b'369.5'
